<a href="https://colab.research.google.com/github/PinkCongolala/Tambakan-ng-mga-kababalaghan/blob/main/Metrics%20Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Working Code for Auto Metrics Extraction

In [ ]:
import os
import shutil
import zipfile
import smartsheet
from google.colab import files
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

ACCESS_TOKEN = "mHm7ovoYkHeuLmYanZPtfqbEBfpHVYMogp4A7"
client = smartsheet.Smartsheet(ACCESS_TOKEN)
client.errors_as_exceptions(True)

# --- CIS sheet mapping (Dominators included) ---
cis_sheets = {
    "Apex": "Apex 2025 Sep",
    "Champions": "Champions 2025 Sep",
    "Dominators": "Dominators 2025 Sep",
    "Stratos": "Stratos 2025 Sep",
    "Kingsmen": "Kingsmen 2025 Sep",
    "The Luminaries": "The Luminaries 2025 Sep",
    "The Royals": "The Royals 2025 Sep",
    "The Elite": "The Elite 2025 Sep",
    "Titans": "Titans 2025 Sep"
}

# --- CMF sheet mapping ---
cmf_sheets = {
    "Apex": "CMF Apex 2025 Sep",
    "Champions": "CMF Champions 2025 Sep",
    "Dominators": "CMF Dominators 2025 Sep",
    "Stratos": "CMF Stratos 2025 Sep",
    "Kingsmen": "CMF Kingsmen 2025 Sep",
    "The Luminaries": "CMF The Luminaries 2025 Sep",
    "The Royals": "CMF The Royals 2025 Sep",
    "The Elite": "CMF The Elite 2025 Sep",
    "Titans": "CMF Titans 2025 Sep"
}

# Temporary folder to store all sheets
temp_dir = "temp_exports"
if os.path.exists(temp_dir):
    shutil.rmtree(temp_dir)
os.makedirs(temp_dir, exist_ok=True)

all_sheets = client.Sheets.list_sheets(include_all=True).data

# --- Function to export sheets into a specific subfolder ---
def export_sheets(sheet_mapping, subfolder, prefix=""):
    folder_path = os.path.join(temp_dir, subfolder)
    os.makedirs(folder_path, exist_ok=True)

    for div, sheet_names in sheet_mapping.items():
        # Ensure it's a list to support multiple sheets
        if not isinstance(sheet_names, list):
            sheet_names = [sheet_names]

        for sheet_name in sheet_names:
            print(f"🔍 Processing {div} - {sheet_name} {prefix}...")

            # Exact-match lookup
            target = next(
                (s for s in all_sheets if sheet_name.lower().strip() == s.name.lower().strip()),
                None
            )

            if not target:
                print(f"⚠️ Sheet not found: {sheet_name}")
                continue

            client.Sheets.get_sheet_as_excel(
                target.id,
                folder_path,
                alternate_file_name=f"{prefix}{sheet_name}.xlsx"
            )
            print(f"✅ Exported {prefix}{sheet_name}")

# --- Export CIS sheets with prefix ---
export_sheets(cis_sheets, subfolder="CIS", prefix="CIS ")

# --- Export CMF sheets without prefix ---
export_sheets(cmf_sheets, subfolder="CMF", prefix="")

# --- Create final zip with subfolders ---
final_zip = "CIS_and_CMF_2025_Sep_Sheets.zip"
with zipfile.ZipFile(final_zip, "w") as zipf:
    for root, _, files_list in os.walk(temp_dir):
        for file in files_list:
            full_path = os.path.join(root, file)
            # Preserve subfolder structure
            arcname = os.path.relpath(full_path, temp_dir)
            zipf.write(full_path, arcname)

# --- Cleanup temp folder ---
shutil.rmtree(temp_dir)

print(f"🎉 All done! Files zipped into {final_zip}")
files.download(final_zip)
